### Установка (при необходимости раскомментить)

In [1]:
# # Uninstall previous version of WhiteBox
# !pip3 uninstall --yes autowoe
# # install requirements if Linux 
# # !pip3 install -r requirements.txt
# # Install WhiteBox
# !python3 setup.py install --user

In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb

from collections import OrderedDict
from sklearn.metrics import roc_auc_score
from copy import deepcopy

from autowoe import ReportDeco, AutoWoE

### Чтение  выборок

In [3]:
data = pd.read_csv("./example_data/data_cat.csv")

In [4]:
train = data.iloc[:14000, :]
test = data.iloc[14000:, :]

### Параметры модели

Для обучения модели рекомендуется указать тип признаков для обучения.
Поэтому создается словарь features_type с ключами: 

 "real" -- вещественный признак
 
 "cat" --  категориальный.
 
 "date"-- ("%Y%d%m", ("m", "d", "wd", "h", "min"))
 
 Для признаков, которые не размечены, типы будут определены автоматом. Такой вариант будет работать, но качество порядочно просядет. __Однако в этот раз воспользуемся этой опцией и дикт features_type создавать не будем__

In [5]:
# подробно параметры описаны в Example_1
auto_woe = AutoWoE(monotonic=False,
                     max_bin_count=5,
                     oof_woe=True,
                     regularized_refit=True,
                     p_val=0.05,
                     debug=False
        )
auto_woe = ReportDeco(auto_woe)

In [6]:
autowoe_fit_params = {"train": train, 
                      "target_name": "isFraud",
}

auto_woe.fit(**autowoe_fit_params)

[autowoe.lib.selectors.selector_first] [INFO]  features [] contain too many nans or identical values


/usr/local/lib/python3.6/dist-packages/lightgbm/basic.py:1291: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')


Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[52]	val_set's auc: 0.886183
[autowoe.lib.selectors.selector_first] [INFO]  features ['ProductCD', 'card4', 'addr2', 'C3', 'D7', 'D12', 'V1', 'V2', 'V3', 'V4', 'V5', 'V7', 'V8', 'V9', 'V14', 'V15', 'V16', 'V17', 'V18', 'V21', 'V22', 'V27', 'V28', 'V31', 'V32', 'V33', 'V34', 'V41', 'V42', 'V44'] have low importance
[autowoe.lib.autowoe] [INFO] TransactionDT processing...
[autowoe.lib.autowoe] [INFO] card1 processing...[autowoe.lib.autowoe] [INFO] card3 processing...

[autowoe.lib.autowoe] [INFO] card6 processing...
[autowoe.lib.autowoe] [INFO] dist1 processing...
[autowoe.lib.autowoe] [INFO] P_emaildomain processing...[autowoe.lib.autowoe] [INFO] C1 processing...

[autowoe.lib.autowoe] [INFO] C4 processing...[autowoe.lib.autowoe] [INFO] C6 processing...[autowoe.lib.autowoe] [INFO] C8 processing...


[autowoe.lib.autowoe] [INFO] dist2 processing...
[autowoe.lib.autowoe] [INFO] C10 processing..

In [7]:
pred = auto_woe.predict_proba(test)
roc_auc_score(test[autowoe_fit_params["target_name"]], pred)

0.8263786316811411

##### Замечание
ReportDeco - обертка для построения отчета. Она не обязательна для обучения и применения модели, но обязательна для построения отчета (см последнюю ячейку). Для доступа к атрибутам самой модели необходимо обратится к атрибуту auto_woe._ReportDeco__auto_woe декоратора

### Результат работы автотипизатора

In [8]:
auto_woe.private_features_type

{'TransactionDT': 'real',
 'TransactionAmt': 'real',
 'card1': 'real',
 'card2': 'real',
 'card3': 'real',
 'card5': 'real',
 'card6': 'cat',
 'addr1': 'real',
 'dist1': 'real',
 'dist2': 'real',
 'P_emaildomain': 'cat',
 'R_emaildomain': 'cat',
 'C1': 'real',
 'C2': 'real',
 'C4': 'real',
 'C5': 'real',
 'C6': 'real',
 'C7': 'real',
 'C8': 'real',
 'C9': 'real',
 'C10': 'real',
 'C11': 'real',
 'C12': 'real',
 'C13': 'real',
 'C14': 'real',
 'D1': 'real',
 'D2': 'real',
 'D3': 'real',
 'D4': 'real',
 'D5': 'real',
 'D6': 'real',
 'D8': 'real',
 'D9': 'real',
 'D10': 'real',
 'D11': 'real',
 'D13': 'real',
 'D14': 'real',
 'D15': 'real',
 'M1': 'cat',
 'M2': 'cat',
 'M3': 'cat',
 'M4': 'cat',
 'M5': 'cat',
 'M6': 'cat',
 'M7': 'cat',
 'M8': 'cat',
 'M9': 'cat',
 'V6': 'real',
 'V10': 'real',
 'V11': 'real',
 'V12': 'real',
 'V13': 'real',
 'V19': 'real',
 'V20': 'real',
 'V23': 'real',
 'V24': 'real',
 'V25': 'real',
 'V26': 'real',
 'V29': 'real',
 'V30': 'real',
 'V35': 'real',
 'V36

### Формирование отчета

In [9]:
report_params = {"automl_date_column": "report_month", # колонка с датой в формате params['datetimeFormat']
                 "output_path": "./AUTOWOE_REPORT_3", # папка, куда сгенерится отчет и сложатся нужные файлы
                 "report_name": "___НАЗВАНИЕ ОТЧЕТА___",
                 "report_version_id": 1,
                 "city": "Воронеж",
                 "model_aim": "___ЦЕЛЬ ПОСТРОЕНИЯ МОДЕЛИ___",
                 "model_name": "___НАЗВАНИЕ МОДЕЛИ___",
                 "zakazchik": "___ЗАКАЗЧИК___",
                 "high_level_department": "___ПОДРАЗДЕЛЕНИЕ___",
                 "ds_name": "___РАЗРАБОТЧИК МОДЕЛИ___",
                 "target_descr": "___ОПИСАНИЕ ЦЕЛЕВОГО СОБЫТИЯ___",
                 "non_target_descr": "___ОПИСАНИЕ НЕЦЕЛЕВОГО СОБЫТИЯ___"}

auto_woe.generate_report(report_params)

No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
No handles with labels found to put in legend.
/mnt/alex/Whitebox_git/autowoe/lib/report/utilities_images/u

[autowoe.lib.report.report_generator] [INFO] Successfully wrote ./AUTOWOE_REPORT_3/autowoe_report.html.
